In [119]:
# !unzip '/content/drive/MyDrive/Pneumonia Pytorch/archive.zip' -d '/content'

In [120]:
# imports
import torch
import torchvision
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torchvision.utils import make_grid
from tqdm import tqdm
from torchvision import transforms, datasets


In [121]:
dir = "/content/chest_xray/chest_xray"


In [122]:
# define transforms and dataset
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
    ])


transform_val = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
    ])

train_dataset = datasets.ImageFolder(
    os.path.join(dir, 'train/'), transform=transform_train)

val_dataset = datasets.ImageFolder(
    os.path.join(dir, 'val/'), transform=transform_val)

test_dataset = datasets.ImageFolder(
    os.path.join(dir, 'test/'), transform=transform_val)

In [123]:
# define data loaderes
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size = 64, shuffle=True)

val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size = 64, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size = 64, shuffle=True)

In [124]:
images, gts = next(iter(train_loader))
print(images.shape)
print(gts.shape)

torch.Size([64, 3, 224, 224])
torch.Size([64])


In [125]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = torchvision.models.densenet121(pretrained=True).to(device)

features = model.classifier.in_features
model.classifier = torch.nn.Linear(features, 2).to(device)

criterion = torch.nn.CrossEntropyLoss().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
epoch = 5

best_loss = None

for i in range(epoch):
  loss = 0

  # training loop
  for inputs,gts in train_loader:
    model.train()
    inputs = inputs.to(device)
    gts = gts.to(device)
    optimizer.zero_grad() 
    out = model(inputs)
    loss = criterion(out, gts)
    loss.backward() 
    optimizer.step() 

    loss += loss.item()

  # save best loss checkpoint
  if best_loss is None:
    torch.save(model.state_dict(), f'/content/drive/MyDrive/pneumonia.pt') 
    print('Saving Model')
    best_loss = loss
  elif best_loss > loss:
    torch.save(model.state_dict(), f'/content/drive/MyDrive/pneumonia.pt') 
    print('Saving Model')
    best_loss = loss

  print('Epoch ' + str(i+1) + ', Loss: ' + str(loss.item()))

In [127]:
# load best model and run it on test set
model.load_state_dict(torch.load('/content/drive/MyDrive/pneumonia.pt'))    
all = 0
correct = 0
with torch.no_grad():
  for inputs , gts in test_loader:
      model.eval()
      inputs = inputs.to(device)
      gts = gts.to(device)
      out = model(inputs)
      pred = torch.max(out, 1)[1]
      correct += pred.eq(gts.data.view_as(pred)).sum()


print("Total number of Test Images : " + str(len(test_loader.dataset)))
print("Accuracy : " + str((correct/len(test_dataset) * 100).item()) )


Total number of Test Images : 624
Accuracy : 91.9871826171875
